##  Setup the environment variables and import dependencies

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *
from openvino.inference_engine import IEPlugin, IENetwork
import cv2
# For labeling the image
from out_process import placeBoxes


### Create an Intermediate Representation of the Model
Model Optimizer creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe*, TensorFlow*, MXNet*, ONNX*, and Kaldi* models are supported by Model Optimizer.

We will use the YOLOv3 model. Download the model. All YOLO models are originally implemented in the Darknet framework and consists of two files: .cfg file with model configurations and .weights file with model weights.

In [ ]:
! pip3 install tensorflow==1.12

In [ ]:
! git clone https://github.com/mystic123/tensorflow-yolo-v3.git

In [ ]:
! cd tensorflow-yolo-v3

### Download coco.names file from the Darknet website or use labels that fit your task

In [ ]:
! wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

### Download the yolov3.weights. 

In [ ]:
! wget https://pjreddie.com/media/files/yolov3.weights

### Run a converter to freeze the graph. 

In [ ]:
!  python3 tensorflow-yolo-v3/convert_weights_pb.py --class_names coco.names --data_format NHWC --weights_file yolov3.weights

## Optimize a deep-learning model using the Model Optimizer (MO) 

In this section, you will use the Model Optimizer to convert a trained model to two Intermediate Representation (IR) files (one .bin and one .xml). The Inference Engine requires this model conversion so that it can use the IR as input and achieve optimum performance on Intel® hardware.



### 1. Create a directory to store IR files

In [ ]:
! mkdir -p tensorflow-yolo-v3/FP32
! mkdir -p tensorflow-yolo-v3/FP16

### 2. Run the Model Optimizer on the frozen YOLOv3 TensorFlow* model. This step generates one .xml file and one .bin file and place both files in the tutorial samples directory (located here: /object-detection/tensorflow-yolo-v3/FP32/)

In [ ]:
!python3 mo_tf.py --input_model frozen_darknet_yolov3_model.pb --batch 1 --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3.json -o tensorflow-yolo-v3/FP16 --data_type FP16

In [ ]:
!python3 mo_tf.py --input_model frozen_darknet_yolov3_model.pb --batch 1 --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3.json -o tensorflow-yolo-v3/FP32

### 3. Verify creation of the optimized model files (the IR files)

In [ ]:
! ls  tensorflow-yolo-v3/FP16

## Use the YOLOv3 model and Inference Engine in an object detection application 

###  Use the sample app (object_detection_demo_yolov3_async.py) from the Intel® Distribution of OpenVINO™ toolkit.

In [ ]:
! python3 object_detection_demo_yolov3_async.py -h

### Source your environmental variables 

In [ ]:
%%bash
source /opt/intel/openvino/bin/setupvars.sh

In [ ]:
os.environ["VIDEO"] = "cars_1900.mp4"

## Create a Job File 

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

To pass the specific variables to the Python code, we will use following arguments:


* `-m`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the optimized models XML
* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-o`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* `-l`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;path to cpu extension library

The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile object_detection_job.sh

ME=`basename $0`

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
while getopts 'd:f:i:r:n:?' OPTION; do
    case "$OPTION" in
    d)
        DEVICE=$OPTARG
        echo "$ME is using device $OPTARG"
      ;;

    f)
        FP_MODEL=$OPTARG
        echo "$ME is using floating point model $OPTARG"
      ;;

    i)
        INPUT_FILE=$OPTARG
        echo "$ME is using input file $OPTARG"
      ;;
    r)
        RESULTS_BASE=$OPTARG
        echo "$ME is using results base $OPTARG"
      ;;
    n)
        NUM_INFER_REQS=$OPTARG
        echo "$ME is running $OPTARG inference requests"
      ;;
    esac  
done


RESULTS_PATH="${RESULTS_BASE}"
mkdir -p $RESULTS_PATH
echo "$ME is using results path $RESULTS_PATH"

    
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 object_detection_demo_yolov3_async.py                        -m tensorflow-yolo-v3/${FP_MODEL}/frozen_darknet_yolov3_model.xml \
                                            -i $INPUT_FILE \
                                            -d $DEVICE \
                                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

### Job queue submission

Each cell below will submit a job to different edge compute nodes. The output of the cell is the JobID of your job, which you can use to track progress of a job.

Note You can submit all 5 jobs at once or follow one at a time.

After submission, they will go into a queue and run as soon as the requested compute resources become available. (tip: shift+enter will run the cell and automatically move you to the next cell. So you can hit shift+enter multiple times to quickly run multiple cells)

### Intel® CPU
In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500TE. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:i5-6500te -F "-r results/CPU -d CPU -f FP32 -i $VIDEO -n 2" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    
    progressIndicator('results/CPU', 'post_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Yolov3 (Intel CPU)',
           ['results/CPU/cars.mp4']
        )

### Intel® Core CPU with Intel® GPU

In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500TE. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU -d GPU -f FP32 -i $VIDEO -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:


    progressIndicator('results/GPU', 'post_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Yolov3 (Intel GPU)',
           ['results/GPU/cars.mp4']
        )

### Intel® Neural Compute Stick 2

In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500te CPU. The inference workload will run on an Intel Neural Compute Stick 2 installed in this node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "-r results/NCS2 -d MYRIAD -f FP16 -i $VIDEO -n 8" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
   
    progressIndicator('results/MYRIAD', 'post_progress.txt', "Inferencing", 0, 100)

In [ ]:
videoHTML('Yolov3 (Intel MYRIAD)',
           ['results/MYRIAD/cars.mp4']
        )